In [ ]:
# --- BAGIAN 0: FUNGSI SETUP DAN IMPORT ---
import os
import sqlite3
from datetime import datetime
import torch
import cv2
from ultralytics import YOLO
from flask import Flask, render_template, request, redirect, url_for, flash, session, jsonify
from werkzeug.utils import secure_filename
from passlib.hash import sha256_crypt
from google.colab import drive
from pyngrok import ngrok, conf
import getpass

print("Semua pustaka berhasil diimpor.")

In [ ]:
# --- BAGIAN 1: KONEKSI GOOGLE DRIVE & PERSIAPAN LINGKUNGAN ---

print("Menghubungkan Google Drive...")
drive.mount('/content/drive')
print("Google Drive terhubung.")

# Ganti 'ProjekYOLOv8_Cloud' jika nama folder Anda berbeda
project_path = '/content/drive/MyDrive/ProjekYOLOv8_Cloud' 
os.chdir(project_path)
print(f"Direktori kerja diubah ke: {os.getcwd()}")

print("\nMenginstal dependensi dari requirements.txt...")
!pip install -q -r requirements.txt
print("Instalasi dependensi selesai.")

In [ ]:
# --- BAGIAN 2: LOGIKA DATABASE ---
DB_FILE = "db_projek_yolo8.db"
SCHEMA_FILE = "schema.sql"

def initialize_database():
    if not os.path.exists(DB_FILE) and os.path.exists(SCHEMA_FILE):
        print(f"Membuat database dari '{SCHEMA_FILE}'...")
        conn = sqlite3.connect(DB_FILE)
        with open(SCHEMA_FILE, 'r') as f:
            conn.executescript(f.read())
        conn.commit()
        conn.close()
        print("Database berhasil dibuat.")
    else:
        print("Database sudah ada atau file skema tidak ditemukan.")

def get_db_connection():
    conn = sqlite3.connect(DB_FILE)
    conn.row_factory = sqlite3.Row
    return conn

# Panggil fungsi inisialisasi database saat startup
initialize_database()


In [ ]:
# --- BAGIAN 3: LOGIKA MODEL AI ---
MODEL_PATH = 'models_yolo/best.pt'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Menggunakan perangkat: {device}")
print(f"Memuat model dari: {MODEL_PATH}...")
model = YOLO(MODEL_PATH)
model.to(device)
print("Model berhasil dimuat.")

def detect_objects_on_image(image_path, user_id):
    try:
        results = model.predict(image_path, conf=0.5)
        frame = cv2.imread(image_path)
        detected_objects = []
        for r in results:
            for box in r.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cls = model.names[int(box.cls[0])]
                conf = float(box.conf[0])
                detected_objects.append(cls)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f"{cls}: {conf:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        ts = datetime.now().strftime("%Y%m%d_%H%M%S")
        out_fn = f"{ts}_{os.path.basename(image_path)}"
        cv2.imwrite(os.path.join('app/static/detections', out_fn), frame)
        
        count = len(detected_objects)
        types = ', '.join(sorted(list(set(detected_objects)))) or "Tidak ada"
        
        conn = get_db_connection()
        conn.execute(
            "INSERT INTO detections (user_id, image_path, detected_objects, object_count, detection_time) VALUES (?, ?, ?, ?, ?)",
            (user_id, out_fn, types, count, datetime.now())
        )
        conn.commit()
        conn.close()
        return out_fn, types, count
    except Exception as e:
        return None, f"Kesalahan: {e}", 0

In [ ]:
# --- BAGIAN 4: DEFINISI LENGKAP APLIKASI FLASK ---
app = Flask(__name__, template_folder='app/templates', static_folder='app/static')
app.config['SECRET_KEY'] = 'gantidenganyangaman'
app.config['UPLOAD_FOLDER'] = 'app/static/uploads'
os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)
os.makedirs('app/static/detections', exist_ok=True)
print("Aplikasi Flask dikonfigurasi.")

# === Rute-rute Flask ===
@app.route('/')
def index(): return render_template('index.html')

@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form['username']
        password_candidate = request.form['password']
        conn = get_db_connection()
        result = conn.execute("SELECT * FROM users WHERE username = ?", (username,)).fetchone()
        conn.close()
        if result and sha256_crypt.verify(password_candidate, result['password']):
            session['logged_in'], session['username'], session['user_id'] = True, result['username'], result['id']
            return redirect(url_for('dashboard'))
        else:
            flash('Username atau password salah.', 'danger')
    return render_template('login.html')

@app.route('/dashboard')
def dashboard():
    return render_template('dashboard.html') if 'logged_in' in session else redirect(url_for('login'))

@app.route('/detect', methods=['POST'])
def detect():
    if 'logged_in' not in session: return redirect(url_for('login'))
    if 'file' not in request.files or not request.files['file'].filename:
        flash('Tidak ada file dipilih.', 'danger')
        return redirect(url_for('dashboard'))
    
    file = request.files['file']
    filepath = os.path.join(app.config['UPLOAD_FOLDER'], secure_filename(file.filename))
    file.save(filepath)
    
    output_filename, objects, count = detect_objects_on_image(filepath, session['user_id'])
    if output_filename:
        return render_template('hasil.html', filename=output_filename, objects=objects, count=count)
    else:
        flash(f'Terjadi kesalahan saat deteksi: {objects}', 'danger')
        return redirect(url_for('dashboard'))

@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        hashed_password = sha256_crypt.encrypt(password)
        conn = get_db_connection()
        try:
            conn.execute("INSERT INTO users (username, password) VALUES (?, ?)", (username, hashed_password))
            conn.commit()
            flash('Registrasi berhasil! Silakan login.', 'success')
            return redirect(url_for('login'))
        except sqlite3.IntegrityError: # Username sudah ada
            flash('Username sudah digunakan.', 'danger')
        finally:
            conn.close()
    return render_template('register.html')

@app.route('/logout')
def logout():
    session.clear()
    flash('Anda telah logout.', 'success')
    return redirect(url_for('login'))

@app.route('/history')
def history():
    if 'logged_in' not in session:
        return redirect(url_for('login'))
    conn = get_db_connection()
    detections = conn.execute(
        "SELECT image_path, detected_objects, object_count, detection_time FROM detections WHERE user_id = ? ORDER BY detection_time DESC", 
        (session['user_id'],)
    ).fetchall()
    conn.close()
    return render_template('history.html', detections=detections)

# Placeholder untuk rute unggah dari ESP (misalnya ESP32-CAM)
@app.route('/upload_from_esp', methods=['POST'])
def upload_from_esp():
    # Implementasi ini akan sangat bergantung pada bagaimana ESP mengirim data.
    # Biasanya, ESP akan mengirim gambar sebagai data multipart/form-data.
    # Anda mungkin perlu menambahkan autentikasi (misalnya, API key) untuk endpoint ini.
    if 'imageFile' not in request.files:
        return jsonify({'error': 'Tidak ada file gambar'}), 400
    
    file = request.files['imageFile']
    if file.filename == '':
        return jsonify({'error': 'Nama file kosong'}), 400

    # Asumsikan ada user_id default atau cara lain untuk mengidentifikasi sumber ESP
    # Untuk contoh ini, kita bisa hardcode user_id atau menerimanya sebagai parameter lain.
    esp_user_id = 1 # Ganti dengan logika yang sesuai
    
    filename = secure_filename(f"esp_{datetime.now().strftime('%Y%m%d_%H%M%S')}_{file.filename}")
    filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
    file.save(filepath)
    
    output_filename, objects, count = detect_objects_on_image(filepath, esp_user_id)
    if output_filename:
        return jsonify({'status': 'sukses', 'filename': output_filename, 'objects': objects, 'count': count}), 200
    else:
        return jsonify({'error': f'Kesalahan deteksi: {objects}'}), 500


In [ ]:
# --- BAGIAN 5: MENJALANKAN SERVER DENGAN NGROK ---
print("\n--- PERSIAPAN NGROK ---")
conf.get_default().log_level = "ERROR"
print("Silakan masukkan Authtoken Ngrok Anda:")
authtoken = getpass.getpass()

if not authtoken:
    print("\nAuthtoken tidak boleh kosong!")
else:
    try:
        ngrok.set_auth_token(authtoken)
        public_url = ngrok.connect(5000, "http")
        print("\n==============================================================")
        print("Aplikasi Anda sekarang dapat diakses secara publik!")
        print(f"URL Publik: {public_url}")
        print("==============================================================")
        # Menjalankan aplikasi Flask. `use_reloader=False` penting di Colab.
        app.run(port=5000, use_reloader=False)
    except Exception as e:
        print(f"\nTerjadi kesalahan: {e}. Pastikan Authtoken Anda benar.")


In [ ]:
# ==============================================================================
# SEL 6: MENJALANKAN APLIKASI FLASK DENGAN NGROK
# ==============================================================================
from pyngrok import ngrok, conf
import getpass # Untuk input token yang lebih aman

# Nonaktifkan pesan peringatan default dari pyngrok
conf.get_default().log_level = "ERROR"

# Minta pengguna untuk memasukkan authtoken ngrok mereka
# Menggunakan getpass membuat input tidak terlihat saat diketik
print("Silakan masukkan Authtoken Ngrok Anda:")
authtoken = getpass.getpass()

if not authtoken:
    print("\nAuthtoken tidak boleh kosong!")
else:
    try:
        # Setel authtoken
        ngrok.set_auth_token(authtoken)

        # Mulai terowongan ngrok ke port 5000 (port default Flask)
        # Protokol http digunakan untuk web server
        public_url = ngrok.connect(5000, "http")

        print("\n==============================================================")
        print("Aplikasi Anda sekarang dapat diakses secara publik!")
        print(f"URL Publik (klik untuk membuka): {public_url}")
        print("==============================================================")
        print("Server Flask sedang berjalan. Tekan tombol STOP di samping sel ini untuk menghentikan.")

        # Jalankan aplikasi Flask
        # 'use_reloader=False' penting di Colab untuk menghindari error
        app.run(port=5000, use_reloader=False)

    except Exception as e:
        print(f"\nTerjadi kesalahan saat memulai Ngrok atau Flask: {e}")
        print("Pastikan Authtoken Anda benar dan coba lagi.")

